# Getting Started with FFTBoost

This notebook provides a quick, hands-on introduction to the `fftboost` library. The goal is to run a complete evaluation: configure, execute cross-validation, and read the statistical acceptance result (J-Beats).

By the end you will:
1. Configure an `fftboost` evaluation.
2. Run the cross-validation harness via the Python API.
3. Inspect the J-Beats acceptance telemetry.
4. (Optional) Run the same config with the CLI.

## Setup: installation and imports

Install from GitHub and import the public API used below.

In [ ]:
# If running in a clean environment, uncomment to install:
# !pip install -q git+https://github.com/pinballsurgeon/fftboost.git

import json
import numpy as np
try:
    from fftboost.api import FFTBoost, load_config_from_yaml
except Exception as e:
    raise RuntimeError(
        "Import failed. Install with: !pip install -q git+https://github.com/pinballsurgeon/fftboost.git"
    ) from e

## Step 1: configuration

Create a compact YAML config for a fast run. The settings mirror typical defaults but use shorter duration and fewer folds for quick execution in notebooks.

In [ ]:
%%writefile demo_config.yaml
seed: 1337
fs: 1000
duration_s: 20          # shorter runtime for demo
window_s: 0.4
hop_s: 0.2
cv_folds: 3             # fewer folds for speed

use_wavelets: true
wavelet_family: "db4"
wavelet_level: 4
use_hilbert_phase: true

coherence_subbands:      # coarse coherence summarization (optional)
  - [1, 40]
  - [40, 100]

target:
  ehi_weights: { thd: 0.45, ipr: 0.55 }

fftboost:
  atoms: 5
  lambda_hf: 0.10
  lambda_coh: 3.0
  min_sep_bins: 3
  ridge_alpha: 0.1

## Step 2: run the evaluation (Python API)

Load the YAML, initialize `FFTBoost`, and run the cross-validation harness. The data generator uses a reproducible synthetic signal suitable for a quick check of the pipeline.

In [ ]:
print("loading configuration…")
config = load_config_from_yaml("demo_config.yaml")

print("initializing model…")
model = FFTBoost(config)

print("running cross-validation…")
model.run_evaluation_with_generator()

print("done.")

## Step 3: interpret J-Beats telemetry

`j_beats_pass` indicates whether the lower bound of the 95% confidence interval on the mean ΔR² is > 0. The other fields provide the mean delta and its CI.

In [ ]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, np.bool_):
            return bool(obj)
        return super().default(obj)

telemetry = model.get_j_beats_telemetry()
print(json.dumps(telemetry, indent=2, cls=NumpyEncoder))

## Step 4 (optional): run the CLI

You can execute the same configuration from a terminal. In notebooks, leave this commented unless your environment exposes the `fftboost` console script on PATH.

In [ ]:
# !fftboost --config demo_config.yaml

## Next steps

- Adjust `atoms`, `lambda_hf`, `lambda_coh`, and `min_sep_bins` to control selection behavior.
- Increase `cv_folds` and `duration_s` for more stable estimates.
- For integration, export the compact model artifact (selected bins, scalers, linear head) and call the single-window inference path in streaming code.